In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp
import math

In [4]:
class Flow:
    def __init__(self,gb,parameter):
        self.gb=gb
        self.param=parameter
    
    def set_data(self):
        
        aperture=self.param["aperture"]
        fracture_perm=self.param["fracture_perm"]
        kx=self.param["perm"]
        f_lambda=self.param["f_lambda"]
        bc_lambda=self.param["bc_lambda"]
        for g, d in self.gb:
            specific_volumes = np.power(aperture, self.gb.dim_max()-self.g.dim)
            # Permeability
            k = kx*np.ones(g.num_cells) * specific_volumes#è la kx e basta per la frattura
            if g.dim < self.gb.dim_max():#la g è quella della frattura?
                k *= fracture_perm
            perm = pp.SecondOrderTensor(k)                   
            
            f=self.set_source(f_lambda,g,specific_volumes)
            
            bc,bc_val=self.set_bc(bc_lambda,g)

            parameters = {"second_order_tensor": perm, "source": f, "bc": bc, "bc_values": bc_val}
            pp.initialize_data(g, d, "flow", parameters)
            
            for e, d in gb.edges():
                mg = d["mortar_grid"]
                # Division through aperture/2 may be thought of as taking the gradient, i.e.
                # dividing by the distance from the matrix to the center of the fracture.
                kn = fracture_perm / (aperture/2)
                pp.initialize_data(mg, d, "flow", {"normal_diffusivity": kn})
    
    def set_bc(self,bc_lambda,g):
        b_faces=g.tags["domain_boundary_faces"].nonzero()[0]
        bc_val=np.zeros(g.num_faces)
        b_face_centers=g.face_centers[:,b_faces]
        
        tol=1e-5
        
        b_inflow = b_face_centers[0, :] < tol
        b_outflow = b_face_centers[0, :] > g.nodes[0][-1]-tol
        
        for i in range(b_faces.size):
            bc_val[b_faces[i]]=bc_lambda(b_face_centers[0,i],b_face_centers[1,i],b_face_centers[2,i])
        
        labels = np.array(["neu"] * b_faces.size)
        labels[np.logical_or(b_inflow, b_outflow)] = "dir"
        bc = pp.BoundaryCondition(g, b_faces,labels)
        
        return bc,bc_val
        
        
        
    def set_source(self,f_lambda,g,specific_volumes):
        f=np.zeros(self.Nx)
        for i in range(g.num_cells):
            f[i]=g.cell_volumes[i]*f_lambda(g.cell_centers[0,i],g.cell_centers[1,i],g.cell_centers[2,i])
        return f
        
    
    def discretize(self):
        method=self.method["method"]
        if(method=="Tpfa"):
            print("Tpfa")
            flow_discretization = pp.Tpfa("flow")
        elif(method=="Mpfa"):
            flow_discretization = pp.Mpfa("flow")
        elif(method=="MVEM"):
            print(self.method)
            flow_discretization = pp.MVEM("flow")
        
        source_discretization = pp.ScalarSource(data_key)
        for g, d in gb:
            d[pp.PRIMARY_VARIABLES] = {"pressure": {"cells": 1}}#gradi libertà, per misti "faces":1
            d[pp.DISCRETIZATION] = {"pressure": {"diffusive": flow_discretization,
                                             "source": source_discretization}}
        
        flow_coupling_discretization = pp.RobinCoupling(data_key, flow_discretization)
        for e, d in gb.edges():
            g1, g2 = gb.nodes_of_edge(e)
            d[pp.PRIMARY_VARIABLES] = {"mortar_flux": {"cells": 1}}
            d[pp.COUPLING_DISCRETIZATION] = {
                "lambda": {
                    g1: ("pressure", "diffusive"),
                    g2: ("pressure", "diffusive"),
                    e: ("mortar_flux", flow_coupling_discretization),
                }
            }
    
    def solve(self):
        if(self.method=="Tpfa"):
            print("Tpfa")
            flow_discretization = pp.Tpfa("flow")
        elif(self.method=="Mpfa"):
            flow_discretization = pp.Mpfa("flow")
        elif(self.method=="MVEM"):
            print(self.method)
            flow_discretization = pp.MVEM("flow")
    
        flow_discretization.discretize(self.g, self.data)
        A, b_flow = flow_discretization.assemble_matrix_rhs(self.g, self.data)
        
        self.data[pp.PARAMETERS]["flow"] 
        if(self.method=="MVEM"):
            rhs_discretization = pp.DualScalarSource("flow")
        else:
            rhs_discretization = pp.ScalarSource("flow")
        
        rhs_discretization.discretize(self.g, self.data)
        _, b_rhs = rhs_discretization.assemble_matrix_rhs(self.g, self.data)
        
        if(self.method=="MVEM"):
            up = sps.linalg.spsolve(A, b_flow+b_rhs)
            p=flow_discretization.extract_pressure(self.g, up, self.data)
            self.data[pp.PARAMETERS]["flow"]["darcy_flux"]=flow_discretization.extract_flux(self.g, up, self.data)
        else:
            p = sps.linalg.spsolve(A, b_flow+b_rhs)
        return p
    
    def plot_pressure(self,p):
        pp.plot_grid(self.g,p,figsize=(15,12))
    
    def get_flux(self,p):
        if(self.method!="MVEM"):
            self.data[pp.STATE] = {"pressure": p}
            pp.fvutils.compute_darcy_flux(self.g, data=self.data)
        darcy_flux = self.data[pp.PARAMETERS]["flow"]["darcy_flux"]
        return darcy_flux
    
    def plot_pressure_flux(self,flux,p):
        if(self.method=="MVEM"):
            flow_discretization = pp.MVEM("flow")
            P0u = flow_discretization.project_flux(self.g,flux,self.data)
            pp.plot_grid(self.g, p, P0u * 0.2, figsize=(15, 12))
        else:
            print("For plotting vel and pressure togheter with this function, it is mandatory to use the MVEM method")

        
            
        

    
         
        
        